In [10]:
import json
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    return data

def write_jsonl(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for item in data:
            file.write(json.dumps(item))
            file.write('\n')

In [11]:
import pandas as pd

dataset = "public"
bx1_name = f"{dataset}/BX1_{dataset}_corrected.csv"
bx2_name = f"{dataset}/BX2_{dataset}.csv"
jsonl_name = f"{dataset}/bx_{dataset}.jsonl"
contexts_name = f"{dataset}/contexts_{dataset}.csv"
contexts_jsonl_name = f"{dataset}/contexts_{dataset}.jsonl"
bx1 = pd.read_csv(bx1_name)
bx2 = pd.read_csv(bx2_name)

In [12]:
def finalize_bx2(bx2: pd.DataFrame, bx2_name: str):
    for _, row in bx2.iterrows():
        question: str = row["question"]
        if "Rephrased Question 1" in question:
            extracted_question = question.splitlines()[2].split("Rephrased Question 1:")[-1]
            row["question"] = extracted_question
    bx2.to_csv(bx2_name, index=False)

In [13]:
finalize_bx2(bx2, bx2_name)

In [14]:
import ast
jsonl_data = []
for i in range(len(bx1)):
    benchmark_row = {
        "question_bx1": bx1["question"][i],
        "question_bx2": bx2["question"][i],
        "answer_tables": ast.literal_eval(bx1["answer_tables"][i])
    }
    jsonl_data.append(benchmark_row)

In [15]:
write_jsonl(jsonl_data, jsonl_name)

In [16]:
contexts = pd.read_csv(contexts_name)

In [17]:
contexts_jsonl = []
for i in range(len(contexts)):
    new_row = {
        "table": contexts["table"][i],
        "context": contexts["context"][i],
    }
    contexts_jsonl.append(new_row)

In [18]:
write_jsonl(contexts_jsonl, contexts_jsonl_name)